In [0]:
import pandas as pd

df = pd.read_csv("../data/GSE135055_norm_counts_TPM_GRCh38.p13_NCBI.tsv", sep='\t')
df

In [0]:
# load gene name mapper
df_annot = pd.read_csv("../data/Human.GRCh38.p13.annot.tsv", sep='\t')
gene_dict = dict(zip(df_annot['GeneID'], df_annot['EnsemblGeneID']))

In [0]:
df['gene'] = df['GeneID'].map(gene_dict)
df = df.drop(columns=['GeneID'])
df = df.dropna(subset=['gene'])
cols = ['gene'] + [col for col in df.columns if col != 'gene']
df = df.set_index('gene')
df = df.transpose()
df 

# Load metadata

In [0]:
sra_df = pd.read_csv("../data/SraRunTable_GSE135055.csv")
cols = ['GEO_Accession (exp)', 'age', 'source_name']
sra_df = sra_df[cols]
sra_df = sra_df.set_index('GEO_Accession (exp)')
sra_df

In [0]:
df_concat = pd.concat([df, sra_df], axis=1)
df_concat['heart_failure'] = df_concat['source_name'].apply(lambda x: 1 if x == 'Heart failure' else 0)
df_concat = df_concat.drop(['source_name'], axis=1)
df_concat


# filter genes by high DE

In [0]:
top100_genes = pd.read_csv("../results/top100_genes.txt", header=0)
top100_genes

In [0]:
shared_genes = list(set(top100_genes.iloc[:, 0]) & set(df_concat.columns[:-2]))
len(shared_genes)

In [0]:
df_filtered = df_concat[['heart_failure'] + shared_genes]
df_filtered

# Model

In [0]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_filtered, test_size=0.2, random_state=42, stratify=df_filtered['heart_failure'])

In [0]:
!pip install autogluon

In [0]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd

train_data = TabularDataset(train_df)

# Optimized hyperparameters for small datasets
hyperparameters = {
    # Linear models with strong regularization
    'LR': [
        # Lasso (L1 regularization) - for feature selection
        {'C': 0.001, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 1000},
        {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 1000},
        {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 1000},
        
        # Ridge (L2 regularization) - for handling correlated features
        {'C': 0.001, 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 1000},
        {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs', 'max_iter': 1000},
        
        # ElasticNet (L1 + L2)
        {'C': 0.01, 'penalty': 'elasticnet', 'solver': 'saga', 'l1_ratio': 0.5, 'max_iter': 1000},
        {'C': 0.1, 'penalty': 'elasticnet', 'solver': 'saga', 'l1_ratio': 0.3, 'max_iter': 1000},
        {'C': 0.1, 'penalty': 'elasticnet', 'solver': 'saga', 'l1_ratio': 0.7, 'max_iter': 1000},
    ],
    
    # Random Forest with strong regularization
    'RF': [
        {'n_estimators': 100, 'max_depth': 3, 'min_samples_split': 5, 'min_samples_leaf': 3},
        {'n_estimators': 200, 'max_depth': 4, 'min_samples_split': 7, 'min_samples_leaf': 4},
        {'n_estimators': 50, 'max_depth': 2, 'min_samples_split': 10, 'min_samples_leaf': 5},
    ],
    
    # Extremely Randomized Trees (often more robust than RF)
    'XT': [
        {'n_estimators': 100, 'max_depth': 3, 'min_samples_split': 5},
        {'n_estimators': 200, 'max_depth': 4, 'min_samples_split': 7},
    ],
    

}

# Exclude tree boosting models that need more data
excluded_model_types = ['GBM', 'XGB', 'CAT', 'NN_TORCH', 'KNN']

hyperparameter_tune_kwargs = {
    'num_trials': 15,  # Focused search for small data
    'scheduler': 'local',
    'searcher': 'auto',
}

# Fit the predictor
predictor = TabularPredictor(
    label='heart_failure',
    problem_type='binary',
    eval_metric='roc_auc'
).fit(
    train_data=train_data,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
    excluded_model_types=excluded_model_types,  # Critical: exclude data-hungry models
    num_bag_folds=5,
    num_stack_levels=0,  # Simpler ensemble for small data
    keep_only_best=False,
    time_limit=300,  # 5 minutes is sufficient for these simple models
    presets='optimize_for_deployment',  # Focus on interpretable, fast models
    use_bag_holdout=True
)

In [0]:
# Get the leaderboard with all models and their AUROC scores
leaderboard = predictor.leaderboard()
leaderboard



In [0]:
test_data = TabularDataset(test_df)

# Evaluate on the test set (TRUE performance estimate)
test_performance = predictor.evaluate(TabularDataset(test_data))
test_auc = test_performance['roc_auc']

print("=== FINAL MODEL EVALUATION ===")
print(f"Test Set AUROC: {test_auc:.4f}")
print(f"Test Set Size: {len(test_data)} samples")

# Compare with training performance for overfitting check
train_performance = predictor.evaluate(TabularDataset(train_data))
train_auc = train_performance['roc_auc']

print(f"\n=== OVERFITTING CHECK ===")
print(f"Training AUROC: {train_auc:.4f}")
print(f"Test AUROC: {test_auc:.4f}")


In [0]:
import shap
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

feature_names = shared_genes

# get the best model from autogluon
best_model_name = leaderboard.iloc[0]['model']  # First model in leaderboard is the best
print(f"Best model: {best_model_name}")

# Prepare data for SHAP (use training data)
X_train = train_data[feature_names]

# Create a wrapper function for AutoGluon predictor
def predict_function(X):
    """Wrapper function that SHAP can use"""
    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X, columns=feature_names)
    # For binary classification, return probabilities for positive class
    preds = predictor.predict_proba(X, model=best_model_name)
    if isinstance(preds, pd.DataFrame):
        return preds.iloc[:, 1].values  # Return positive class probabilities
    elif isinstance(preds, np.ndarray):
        return preds[:, 1] if len(preds.shape) > 1 else preds
    return preds

# Use Permutation/Partition explainer (works for any model, reasonably fast)
explainer = shap.Explainer(predict_function, X_train, algorithm="permutation")
shap_values = explainer(X_train)

print(f"SHAP values shape: {shap_values.values.shape}")



In [0]:
# Calculate feature importance
shap_importance = np.abs(shap_values.values).mean(0)
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': shap_importance
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Genes (SHAP):")
print(feature_importance_df.head(10))

# Save feature importance to CSV
feature_importance_df.to_csv('../results/shap_feature_importance.csv', index=False)

# Create visualizations
# Summary plot (beeswarm)
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_train, max_display=20)
plt.tight_layout()
plt.savefig('../results/shap_summary.png', dpi=300, bbox_inches='tight')
plt.close()

# Bar plot (mean absolute SHAP values)
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_train, plot_type="bar", max_display=20)
plt.tight_layout()
plt.savefig('../results/shap_bar.png', dpi=300, bbox_inches='tight')
plt.close()

# Waterfall plot for first sample
plt.figure(figsize=(10, 6))
shap.plots.waterfall(shap_values[0], max_display=15)
plt.tight_layout()
plt.savefig('../results/shap_waterfall_sample0.png', dpi=300, bbox_inches='tight')
plt.close()


# test on dataset unseen by the model

In [0]:
df_test_new = pd.read_csv("../data/GSE123976_norm_counts_TPM_GRCh38.p13_NCBI.tsv", sep='\t')
df_test_new

In [0]:
df_test_new['gene'] =df_test_new['GeneID'].map(gene_dict)
df_test_new =df_test_new.drop(columns=['GeneID'])
df_test_new =df_test_new.dropna(subset=['gene'])
cols = ['gene'] + [col for col in df_test_new.columns if col != 'gene']
df_test_new =df_test_new.set_index('gene')
df_test_new =df_test_new.transpose()
df_test_new 

In [0]:
sra_df = pd.read_csv("../data/SraRunTable_GSE123976.csv")
sra_df

In [0]:
sra_df = pd.read_csv("../data/SraRunTable_GSE123976.csv")
cols = ['GEO_Accession (exp)', 'age', 'diagnosis']
sra_df = sra_df[cols]
sra_df = sra_df.set_index('GEO_Accession (exp)')
sra_df

In [0]:
df_concat_new = pd.concat([df_test_new, sra_df], axis=1)
df_concat_new['heart_failure'] = df_concat_new['diagnosis'].apply(lambda x: 1 if x == 'HF' else 0)
df_concat_new = df_concat_new.drop(['diagnosis'], axis=1)
df_concat_new

In [0]:
df_filtered_new = df_concat_new[['heart_failure'] + list(shared_genes)]
df_filtered_new

In [0]:
unseen_test_data = TabularDataset(df_filtered_new)
unseen_test_performance = predictor.evaluate(unseen_test_data)
unseen_test_performance['roc_auc']

# save the model

In [0]:
# Save the model
predictor.save('../heart_failure_predictor_model')



In [0]:
# Later, load it back
predictor = TabularPredictor.load('../heart_failure_predictor_model')

In [0]:
predictor.save('')
predictor.save_space()
import shutil
shutil.make_archive('../heart_failure_predictor_model', 'zip', '../heart_failure_predictor_model')


In [0]:
# # verify the saved model works
# import pickle
# from autogluon.tabular import TabularDataset, TabularPredictor
# with open('../results/heart_failure_predictor.pkl', 'rb') as f:
#     loaded_predictor = pickle.load(f)
# # Verify it's the same as original
# original_performance = loaded_predictor.evaluate(TabularDataset(test_df))
# print(f"Original model AUROC: {original_performance['roc_auc']:.4f}")